# Implementation - Implicit Scheme Diffusion Only


Define helpful functions to get the steady state solution and to print a progress bar

In [1]:
def get_static_sol(filename):
    file = open(filename, 'r')
    X = []
    Y = []
    for line in file:
        aux1, aux2 = line.split()
        X.append(float(aux1))
        Y.append(float(aux2))
    return X, Y

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 0, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

Next, we need to define the constants for the model. In particular, we need to define the ionic force $\kappa$, which has the following expression

\begin{equation}
\kappa = \sqrt{\frac{(zF)^2}{(RT)^2\epsilon}} = \sqrt{\frac{(ze)^2}{(k_bT)^2\epsilon}}
\end{equation}

Also we need to define the diffusion coefficient $D$.

Since we are working with the dimentionless potential
$$\Psi = \frac{ze}{k_bT}\phi$$ 
the border condition for the potential must be transformed acordingly: $\bar{V_0} = \frac{ze}{k_bT} V_0$

In [2]:
import numpy as np

T = 300
coef = 2 * 1.60217662E-19 / (1.38064852E-23 * T)
V_0 = -coef * 0.15
Cb = 0.1

epsilon = 80.9 * 8.85418782E-12
k = np.sqrt((coef)**2 / epsilon)
D1 = 1.05
D2 = 0.76
print("retrieving files")

x_cp, numCp = np.asarray(get_static_sol('../ch2/results-steadystate/cp-num-r5e-08.txt'))
x_cm, numCm = np.asarray(get_static_sol('../ch2/results-steadystate/cm-num-r5e-08.txt'))
x_psi, numPsi = np.asarray(get_static_sol('../ch2/results-steadystate/potential-num-r5e-08.txt'))


Psi0 = numPsi[0]
print(Psi0)

retrieving files
-11.577098546694161


### Define Mesh Parameters

First, we define the mesh parameters
$$x = [0,\delta]$$
$$t = [0, 1e-2]$$
where x we divide in M intervales and t in N intervals.



In [3]:
import numpy as np
N = 500
M = 1000
length = 1.
Time = 1./10000
dx = length/M
dt = Time/N
Cp = np.zeros([N,M])
Cm = np.zeros([N,M])
Psi  = np.zeros([N,M])
E = np.zeros([N,M])

Now we compute the $\rho = D\frac{dt}{dx^2}$ to check if the mesh is sufficiently small in order for the finite difference method to converge. If $\rho>0,5$ we rise an error.

In [4]:
rho1 = dt * D1 / (dx) ** 2
rho2 = dt * D2 / (dx) ** 2
print(rho1)
print(rho2)
if rho1 > 0.5:
    raise ValueError('the rho1 parameter is greater than the allowed tolerance')
if rho2 > 0.5:
    raise ValueError('the rho2 parameter is greater than the allowed tolerance')

0.21000000000000005
0.15200000000000002


Defining the initial conditions, which are 
$$C_s(0,x) = C_b$$

(well-stired solution) and 
$$\Psi(0,x) = 0$$ 

In [6]:
Cp[0,0:M-1] = Cb #*np.random.rand(M-1)
Cm[0,0:M-1] = Cb #*np.random.rand(M-1)
Psi[0,:] = 0#-np.random.rand(M)
Psi[0,0] = Psi0
Psi[:,M-1] = 0

numCp = Cb
numCm = Cb
numPsi = Psi0


Recall that the Poisson equation is approximated in this scheme as

\begin{eqnarray}
\nabla^2 \Psi = -(C_+ - C_-) = -\Delta C\\
\approx \Psi^{k+1} - 2\Psi^{k} + \Psi^{k-1} = -\Delta x^2 (C_+-C_-)
\end{eqnarray}

The discrete equation forms a vector
\begin{equation}
    \underline{\Psi} = (\Psi^{0}, \Psi^{1}, ..., \Psi^{M-1}).
\end{equation}

Thus, the descrete Poisson equation is writen as a linear problem

\begin{equation}
    A\underline{\Psi} = -\Delta x^2 \Delta \underline{C} + \underline{b}
\end{equation}

Where 

\begin{equation}
    A =\begin{bmatrix}
    -2       & 1  & 0 & \dots & 0   & 0\\
    1       & -2 & 1 & \dots & 0 & 0 \\
    & & & \dots & & &  \\
    & & & \dots & & &  \\
   0       & 0 & 0 & \dots & -2 & 1 \\
   0       & 0 & 0 & \dots & 1 & -2 
\end{bmatrix}
\end{equation}


\begin{equation}
    \underline{b} =\begin{bmatrix}
    -\Psi_0       \\
    0       \\
     \vdots \\
   0    
\end{bmatrix}
\end{equation}

In [7]:
import numpy as np
from numpy.linalg import inv
import scipy as sp
import scipy.sparse
print("creating coefficient matrix")

a = np.ones(M)
b1 = -2 * np.ones(M)
c = np.ones(M) 
positions = [-1, 0, 1]
A = sp.sparse.spdiags(np.array([a, b1, c]), positions, M-3, M-3).todense()
Ainv = np.asarray(np.linalg.inv(A))
print("done creating coefficient matrix")
print("Setting up b vector on the Psi system")
b = np.zeros([N,M])
print("done...")

creating coefficient matrix
done creating coefficient matrix
Setting up b vector on the Psi system
done...


In [8]:
print("System is ready to be solved... starting iteration now")
from scipy.integrate import trapz

Cp[0,M-1] = Cb
Cm[0,M-1] = Cb
Psi[0,M-1] = 0
Psi[0,0] = Psi0
#r = 0.001
sig = -2 * np.tanh(Psi0/4)/(np.tanh(Psi0/4)**2-1)

a = (1 + rho1) * np.ones(M)
b1 = -2 * rho1 * np.ones(M)
c = rho1 * np.ones(M) 
B = sp.sparse.spdiags(np.array([a, b1, c]), positions, M-3, M-3).todense()
for n in range(0,N-1):
    Cp[n+1,M-1] = Cb
    Cm[n+1,M-1] = Cb
    Cp[n+1,M-2] = Cb
    Cm[n+1,M-2] = Cb
    Psi[n+1,0] = Psi0
    Psi[n+1,M-1] = 0
    v1 = -(Psi[n,1:] - Psi[n,:-1])
    v2 = -(Psi[n, 2:] - Psi[n,1:-1])
    positions = [1,2]
    D1 = sp.sparse.spdiags(np.array([v1, v2]), positions, M-3, M-3).todense()
    D2 = sp.sparse.spdiags(np.array([-v1, -v2]), positions, M-3, M-3).todense()
    R1 = B + D1
    R2 = B + D2
    Cp[n+1,:-2] = R1.dot(Cp[n,:-2])
    Cm[n+1,:-2] = R1.dot(Cm[n,:-2])
    Cp[n+1,-2] = rho1 * Cp[n,-2] - rho1 * Cb * (Psi[n, -3] - Psi[n,-2])
    Cm[n+1,-2] = rho2 * Cp[n,-2] + rho2 * Cb * (Psi[n, -3] - Psi[n,-2])
    
    #for j in range(0, M-2):
        #Compute next value of concentrations using previous ones
    #    Cp[n + 1, j] = (1 + rho1) * Cp[n, j] + rho1 * (-2 - (Psi[n, j+1] - Psi[n,j])) * Cp[n,j+1] + rho1 * (1 - (Psi[n, j+2] - Psi[n,j+1])) * Cp[n,j+2]
    #    Cm[n + 1, j] = (1 + rho2) * Cm[n, j] + rho2 * (-2 + (Psi[n, j+1] - Psi[n,j])) * Cm[n,j+1] + rho2 * (1 + (Psi[n, j+2] - Psi[n,j+1])) * Cm[n,j+2]
    #    b[n+1,j] = dx ** 2 * (Cp[n+1,j] - Cm[n+1, j])
    
    b[n+1,:-2] = (Cp[n,1:-1] - Cm[n,1,-1])
    b[n+1,1] = b[n+1,1] + Psi0
    Psi[n+1,1:M-2] = - Ainv.dot(b[n+1,1:M-2])
    
    #for i in range(1, M-1):
    #    E[n+1,i] = trapz(b[n+1,i:])    
    
    #for i in range(1, M-1):
    #    Psi[n+1,i] = trapz(E[n+1,i:])    
    
    
    
    print("\rCompleted: " + "time axis: " + str("{0:.2f}".format((n/(N-2)) * 100)) + " %", end="")
    
    
print('\a')
print("\n... done computing")


System is ready to be solved... starting iteration now


ValueError: number of diagonals (1) does not match the number of offsets (2)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

def f(t):
    x = np.linspace(0, length, M)
    plt.figure(figsize=(20,10))
    plt.subplot(1, 3, 1)
    plt.title('$\Psi$')
    plt.plot(x, Psi[t,:])
    plt.subplot(1, 3, 2)
    plt.ylim(0,10)
    plt.title('$C_+$')
    plt.plot(Cp[t,1:])
    plt.subplot(1, 3, 3)
    plt.ylim(0,10)
    plt.title('$C_-$')
    plt.plot(Cm[t,1:])
    plt.show()


interact(f, t=widgets.IntSlider(min=0,max=N-1,step=1,value=0));